######  <span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#c37235">Bag of words, Tokenizer, and Doc2Vec
<font color=darkblue>
    
    
- This is simple approach of bag of Words, Tokenizer, and Doc2Vec


In [1]:
# Tenorflow 
import tensorflow as tf
import numpy as np

# Tenorflow Padding Sequences
from keras.preprocessing.sequence import pad_sequences

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')

/home/jayanthikishore/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
tweets = ['there is a snake in my boot', 'there is boot snake in my house', 'a a a a']
labels = [75, 12, 50]

In [3]:
tweets = [tweet.split(' ') for tweet in tweets]
unique_words = np.unique(tweets)
unique_words, tweets

(array([list(['a', 'a', 'a', 'a']),
        list(['there', 'is', 'a', 'snake', 'in', 'my', 'boot']),
        list(['there', 'is', 'boot', 'snake', 'in', 'my', 'house'])],
       dtype=object),
 [['there', 'is', 'a', 'snake', 'in', 'my', 'boot'],
  ['there', 'is', 'boot', 'snake', 'in', 'my', 'house'],
  ['a', 'a', 'a', 'a']])

In [4]:
tokenizer = {}

counter = 0
for tweet in tweets:
  for word in tweet:
    if word not in tokenizer:
      tokenizer[word] = counter
      counter += 1

tokenizer

{'there': 0,
 'is': 1,
 'a': 2,
 'snake': 3,
 'in': 4,
 'my': 5,
 'boot': 6,
 'house': 7}

######  <span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#6bc335">Bag of words 
<font color=darkblue>

In [5]:
bag_words = []
# count_count = [0]*len(unique_words)

for tweet in tweets:
  word_count = [0]*(counter)
  
  #Counts instence of every unique word that appears
  for word in tweet:
    locWord = tokenizer[word] # Get the index location of the words
    word_count[locWord] += 1 # Counts the number of times that word appears
    
  # Append after finnished counting
  bag_words.append(word_count)

bag_words

[[1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 0, 1, 1, 1, 1, 1], [0, 0, 4, 0, 0, 0, 0, 0]]

In [6]:
word_count

[0, 0, 4, 0, 0, 0, 0, 0]

In [7]:
# Twitter length
token_len = 50

# Create Decorator
tokenizer = tf.keras.preprocessing.text.Tokenizer() # Sets up the Tikenizer which we will feed

# Fitting the Tokenizer and building our Corpus
tokenizer.fit_on_texts(tweets)

# Create our sequence
X = tokenizer.texts_to_sequences(tweets)

# Padding the text
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=token_len, padding= 'post', truncating='post')

# Convert array to Tensor
X = tf.constant(X, dtype=tf.int64)
y = tf.constant(labels, dtype=tf.int64)

In [8]:
X.shape

TensorShape([3, 50])

In [9]:
y.shape

TensorShape([3])

######  <span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#353ac3">Doc2Vector 
<font color=darkblue>

In [10]:
sampledata = ["Data scientists should know the Mathematics, statistics, and programming",
             "Data scientist familiar in machine learning deep learning and artificial intelligence",
             "Data Scientist should know python and R coding",
             "Data Scientist should know the how to train test and validation of the model",
              "Metrics is more important for model validation"]

tag_sample = [TaggedDocument(words = word_tokenize(dat.lower()), tags =[str(i)]) for i,dat in enumerate(sampledata)]
tag_sample

[TaggedDocument(words=['data', 'scientists', 'should', 'know', 'the', 'mathematics', ',', 'statistics', ',', 'and', 'programming'], tags=['0']),
 TaggedDocument(words=['data', 'scientist', 'familiar', 'in', 'machine', 'learning', 'deep', 'learning', 'and', 'artificial', 'intelligence'], tags=['1']),
 TaggedDocument(words=['data', 'scientist', 'should', 'know', 'python', 'and', 'r', 'coding'], tags=['2']),
 TaggedDocument(words=['data', 'scientist', 'should', 'know', 'the', 'how', 'to', 'train', 'test', 'and', 'validation', 'of', 'the', 'model'], tags=['3']),
 TaggedDocument(words=['metrics', 'is', 'more', 'important', 'for', 'model', 'validation'], tags=['4'])]

In [11]:
# import nltk
# # nltk.download()
# from gensim.models.doc2vec import Doc2Vec, Tagdoc
# from nltk.tokenize import word_tokenize

# data = ["I love machine learning. Its awesome.",
#         "I love coding in python",
#         "I love building chatbots",
#         "they chat amagingly well"]

# tagged_data = [Tagdoc(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
# tagged_data

In [12]:
import multiprocessing
from gensim.models.doc2vec import Doc2Vec
cores = multiprocessing.cpu_count()

doc2vecmodel = Doc2Vec(dm=1, vector_size=20, alpha=0.025,negative=5, hs=0,min_count=1,min_alpha=0.00025,epochs=50)
  
doc2vecmodel.build_vocab(tag_sample)

for epoch in range(doc2vecmodel.epochs):
    print("Iteration number {0}".format(epoch))
    doc2vecmodel.train(tag_sample,total_examples=doc2vecmodel.corpus_count,
                      epochs=doc2vecmodel.epochs)
    #decrease the learning rate
    doc2vecmodel.alpha -=0.0002
    #fix the learning rate, no decay
    doc2vecmodel.min_alpha=doc2vecmodel.alpha
    
#save the model
doc2vecmodel.save("/home/jayanthikishore/Downloads/doc2vec.model")
print("Model Successfully Saved")

Iteration number 0
Iteration number 1
Iteration number 2
Iteration number 3
Iteration number 4
Iteration number 5
Iteration number 6
Iteration number 7
Iteration number 8
Iteration number 9
Iteration number 10
Iteration number 11
Iteration number 12
Iteration number 13
Iteration number 14
Iteration number 15
Iteration number 16
Iteration number 17
Iteration number 18
Iteration number 19
Iteration number 20
Iteration number 21
Iteration number 22
Iteration number 23
Iteration number 24
Iteration number 25
Iteration number 26
Iteration number 27
Iteration number 28
Iteration number 29
Iteration number 30
Iteration number 31
Iteration number 32
Iteration number 33
Iteration number 34
Iteration number 35
Iteration number 36
Iteration number 37
Iteration number 38
Iteration number 39
Iteration number 40
Iteration number 41
Iteration number 42
Iteration number 43
Iteration number 44
Iteration number 45
Iteration number 46
Iteration number 47
Iteration number 48
Iteration number 49
Model Succ

In [13]:
print(doc2vecmodel)

Doc2Vec(dm/m,d20,n5,w5,s0.001,t3)


In [15]:
#Access the saved model file
doc2vecmodel= Doc2Vec.load("/home/jayanthikishore/Downloads/d2v.model")

#to find the vector of a document which is not in the training data
test_line = word_tokenize("Data Scientist calculates metrics for every model")
vec = doc2vecmodel.infer_vector(test_line)
print("Infer: ",vec)

#to find most similar doc using tags
sim_doc = doc2vecmodel.docvecs.most_similar("1")
print(sim_doc)

# otherwise
print(doc2vecmodel.docvecs["1"])

Infer:  [ 0.00600685 -0.01081637 -0.02342382  0.01165458  0.00160417 -0.01647668
 -0.00257421  0.00148563 -0.02041394  0.00884137 -0.01830285 -0.02257978
 -0.01210968  0.00044988  0.02139286 -0.0040398  -0.0010805   0.02480741
 -0.00246424 -0.01426513]
[('3', 0.22874553501605988), ('2', 0.1877303570508957), ('0', 0.013264761306345463)]
[-0.85137886  0.89662504 -1.123845    0.37472183  1.1933655  -1.7735094
 -1.6574339  -2.1300223   1.4840639  -1.8634331   1.7724966   1.2565795
 -0.77021134 -0.4470133   0.07611626  0.38757312 -0.08701865 -2.3551352
 -0.5461885   0.17398375]
